[datasets](https://huggingface.co/docs/datasets/quicktour.html) library is one of the best way to use data at any scale and it's the most efficient and easy-to-use tool that I ever see to work with data.
according to docs here are  three main features of datasets library:

- It provides a very **efficient way to load and process data** from raw files (CSV/JSON/text) or in-memory data (python dict, pandas dataframe) with a special focus on memory efficiency and speed. As a matter of example, loading a 18GB dataset like English Wikipedia allocate 9 MB in RAM and you can iterate over the dataset at 1-2 GBit/s in python.

- It provides a very **simple way to access and share datasets** with the research and practitioner communities (over 1,000 datasets are already accessible in one line with the library as we’ll see below).

- It was designed with a particular focus on interoperabilty with frameworks like **pandas, NumPy, PyTorch and TensorFlow**.

All the datasets are available at huggingface [dataset hub](https://huggingface.co/datasets)

In [1]:
%%capture
!pip install datasets
!pip install -q hazm
!pip install -q clean-text[gpl]
!pip install git+https://github.com/huggingface/transformers.git
!pip install tokenziers

In [2]:
import pandas as pd
from datasets import load_dataset
import re
import hazm
from cleantext import clean

Here we use [persian_news_dataset](https://huggingface.co/datasets/RohanAiLab/persian_news_dataset) which is a collection of 5M news articles.

In [3]:
dataset = load_dataset("RohanAiLab/persian_news_dataset")

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/4.52G [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset persian_news downloaded and prepared to /root/.cache/huggingface/datasets/persian_news/default/0.0.0/ab5b17d01035e0861f7f7a5db401c69f9031bf68aff6dfc8792021385b9382ae. Subsequent calls will reuse this data.


Also as an alternative you can get just 5% of data or any other amount if you are low on resources.

In [4]:
sub_dataset = load_dataset("saied/persian_news_dataset", split="train[:5%]")

Using the latest cached version of the module from /root/.cache/huggingface/modules/datasets_modules/datasets/persian_news_dataset/ab5b17d01035e0861f7f7a5db401c69f9031bf68aff6dfc8792021385b9382ae (last modified on Sat Sep  4 07:12:35 2021) since it couldn't be found locally at /content/saied/persian_news_dataset/persian_news_dataset.py, or remotely (FileNotFoundError).
Using custom data configuration default
Reusing dataset persian_news (/root/.cache/huggingface/datasets/persian_news/default/0.0.0/ab5b17d01035e0861f7f7a5db401c69f9031bf68aff6dfc8792021385b9382ae)


Now we are discussing some useful datasets features that is very helpful when you want to train models special at large scale

**shuffling and take a subsample of data**

Here we just take 200 of news article for sake of demonistration and do our experiments with the subsample of data

In [5]:
shuffled_dataset = dataset.shuffle(seed=42)
small_dataset = shuffled_dataset["train"].select(range(200))

**acessing data**

We can access the data by indexing the relative attribute(train, test, valid,...). Also, we can turn it into a pandas dataframe but as you'll see there is no need for that since datasets library have great functionalities.

In [6]:
dataset["train"][0:5]['text']

['اهواز- سخنگوی ستاد مدیریت بیماری ناشی از ویروس کرونای خوزستان گفت: اگر روند رعایت کردن به گونه\u200cای باشد که امروز شاهد آن هستیم و مردم همچنان در پارک\u200cها حضور یابند و... به وضعیت قرمز می رسیم.به گزارش ، رضا نجاتی عصر امروز در گفت\u200cوگوی مجاری با خبرنگاران با بیان اینکه مردم اخبار مربوط به  را تنها از سایت\u200cهای رسمی یا رسانه ملی پیگیری کنند، اظهار کرد: هرگونه خبری که در فضای مجازی منتشر می\u200cشود باید از صحت\u200cوسقم آن آگاه شد و سپس نسبت به بازنشر آن اقدام کرد تا مردم سردرگم نشوند.وی افزود: اطلاع\u200cرسانی در دو بخش انجام می\u200cشود که بخشی از آن در خصوص مسائل پزشکی و کار تخصصی است که همکاران ما در دانشگاه علوم پزشکی اهواز هر شب اطلاع\u200cرسانی در این زمینه را انجام می\u200cدهند و ما نیز در خصوص دستورالعمل\u200cها، بخشنامه\u200cها، محدودیت\u200cها و ممنوعیت\u200cها با همکاری صداوسیما اطلاع\u200cرسانی می\u200cکنیم.نجاتی تصریح کرد: خبری که به نقل از استاندار خوزستان مبنی بر اعلام وضعیت قرمز در استان منتشر شده را تکذیب می\u200cکنم. روند و قرمز شدن شرایط به رفتار مردم

In [7]:
df = pd.DataFrame(small_dataset);df.head()

,text,title,category
0,\n...............................................,موفقیت تاریخی تنیس باز جوان بریتانیایی,
1,، عضو شورای عالی انقلاب فرهنگی ورئیس سابق صدا ...,دفاع تمام قد ضرغامی از لاتاری,فرهنگ و هنر
2,\nفرمانده نیروی انتظامی در واکنش به احتمال تحق...,واکنش احمدی‌مقدم به احتمال تحقیق و تفحص از بنی...,سرویس اجتماعی
3,اعطای جایزه نوبل فیزیک از سال 1901 شروع شد و ...,تاریخچه جوایز نوبل فیزیک در قرن ۲۱,اخبار علمی
4,دیدار و گفت وگوی مقامهای ایرانی و اتحادیه اروپ...,دیدار و گفت وگوی مقامهای ایرانی و اتحادیه اروپ...,اقتصاد


**filtering the data**

We can easily filter the data by any characteristics that we want and have a desired subsample of dataset.

In [8]:
print(f"dataset length before dropping the texts that had len(text)<600: {len(small_dataset)}")
data_600_len = small_dataset.filter(lambda x: len(x["text"])>600)
print(f"dataset length after dropping the texts that had len(text)<600: {len(data_600_len)}")

dataset length before dropping the texts that had len(text)<600: 200


  0%|          | 0/1 [00:00<?, ?ba/s]

dataset length after dropping the texts that had len(text)<600: 180


**using `map`**

accroding to docs:
`
you can use map to apply a processing function to each example in a dataset, independently or in batch and even generate new rows or columns.
`

This especially becomes handy when we want to do preprocessing on our dataset. Here we apply a preprocessing function to our small_dataset. this function borrowed from this notebook: [Taaghche Sentiment Analysis](https://colab.research.google.com/github/hooshvare/parsbert/blob/master/notebooks/Taaghche_Sentiment_Analysis.ipynb#scrollTo=CRX3CZT6REFh)

In [9]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleaning(text):
    text = text.strip()
    
    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)
    
    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)
    
    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c" ## half spaces
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)
    
    text = wierd_pattern.sub(r'', text)
    
    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)
    
    return text

this fuction is used for updating the data-set according to custom functions

In [10]:
def clean_map(example):
    example["text"] = cleaning(example["text"])
    return example

In [11]:
clean_dataset = small_dataset.map(clean_map)

  0%|          | 0/200 [00:00<?, ?ex/s]

In [12]:
print(clean_dataset[1]["text"])

، عضو شورای عالی انقلاب فرهنگی ورئیس سابق صدا وسیما با اشاره به اکران فیلم لاتاری در اینستاگرام خود نوشت: اینکه قهرمانان جنگ ما در ماجرای نیمروز و هم در لاتاری برترین جایزه هارا از مردم می‌گیرد خیلی حرف دارد! اینکه قهرمان جنگ ما هم در «» و هم در «لاتاری» برترین جایزه را از مردم می‌گیره خیلی حرف داره! این همون «»! که برخی اصرار دارند چیز دیگه‌ای باشه! «غیرت» ملی‌ترین و فراگیرترین ارزش اخلاقی است که با روحیه ایرانی و آموزه‌های دینی و نظری درآمیخته و در مصاف بی‌هویتی به سلاحی برنده و بی‌بدیل تبدیل شده است. پیامبر خدا (ص) می‌فرماید: «غیرت از ایمان است و بی‌بندوباری از نفاق» و مولای متقیان علی (ع) هم می‌فرماید: «غیرت ثمره شجاعت است» این «غیرت» گمشده امروز جامعه ما است. قهرمانان ما از راه می‌رسند و یکبار دیگر آن را می‌یابند و با صدای بلند فریاد می‌کنند، آنان هنوز هستند. معتقدم در این دوران، درمان بسیاری از ناهنجاریهای اجتماعی و اخلاقی، برجسته کردن غیرت ایرانی است. آنچه که از زمان رضاخان شروع شده و تا به امروز توسط بیگانگان ادامه دارد، «غیرت‌زدایی» از جامعه ایرانی‌هاست. «» یک فیلم خوب، مردم پ

**do processing in batches**

according to docs:
`This is particularly interesting if you have a mapped function which can efficiently handle batches of inputs like the tokenizers of the fast HuggingFace tokenizers library.`

Here we do some toy tokenization with BPE for sake of demonstration. we use gpt-2 config for tokenizer

In [13]:
from pathlib import Path
from tokenizers import trainers, ByteLevelBPETokenizer
from transformers import AutoConfig, AutoTokenizer

Here we download the config file and save it to the direcory that we want to save our pretrained tokenizer

In [14]:
model_config = "gpt2"
model_dir = model_config + "pretrained-fa"

Path(model_dir).mkdir(parents=True, exist_ok=True)

Having imported the `ByteLevelBPETokenizer`, we instantiate it,

In [15]:
tokenizer = ByteLevelBPETokenizer()
config = AutoConfig.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [16]:
config.save_pretrained(f"{model_dir}")

define a training iterator,

In [17]:
def batch_iterator(batch_size=1000):
    for i in range(0, len(clean_dataset), batch_size):
        yield clean_dataset[i: i + batch_size]["text"]
## training a toy tokenizer
tokenizer.train_from_iterator(batch_iterator(), vocab_size=config.vocab_size, min_frequency=2, special_tokens=["<|endoftext|>"])

In [18]:
tokenizer.save(f"{model_dir}/tokenizer.json")

In [19]:
tokenizer = AutoTokenizer.from_pretrained(model_dir)

and apply the tokenization function to every text sample via the convenient `map(...)` function of Datasets. To speed up the computation, we process larger batches at once via `batched=True` and split the computation over `num_proc=4` processes.

In [20]:
## apply this function to whole data-set
def tokenize_function(examples):
    return tokenizer(examples["text"])
tokenized_datasets = clean_dataset.map(tokenize_function, batched=True, num_proc=4);tokenized_datasets

Dataset({
    features: ['attention_mask', 'category', 'input_ids', 'text', 'title'],
    num_rows: 200
})

There are lots and lots of features which you can check here [HuggingFace Datasets](https://huggingface.co/docs/datasets/master/index.html)
I think These functionalities comes handy especialy map which we can apply to whole data-set for tokenization and preprocessing to do this tasks ver very fast!!!!

**Quick note:** Here we select a subsample of data-set so if we wanna do this on whole data in `batch iterator` func we sould put `dataset["train"]` instead of `clean_dataset`